# 질문
1. 시작점이 Prediction 스텝인데 1-3 Model Validation Step의 인자값을 불러와서 Predcition Processing한테 줄 수 있는 방법이 없을까요?
2. Quicksight Refresh Now는 왜 업데이트에 왜 한계가 있을까요? `WaiterError: Waiter PipelineExecutionComplete failed: Max attempts exceeded`
3. Sagemaker Processor가 끝나고 Lambda가 실행되게 하려면 어떻게 해야하나요?
4. S3에 데이터가 업데이트가 되는것이 Trigger가 되어서 Sagemaker Processor가 실행되게 하려면 어떻게 해야한아ㅛ?

In [1]:
LOCAL_MODE = False

# 0. 환경설정

In [2]:
import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from pathlib import Path
import joblib

import logging
import logging.handlers

import json
import base64
import boto3
import sagemaker
from botocore.client import Config
from botocore.exceptions import ClientError

import time
from datetime import datetime as dt
import datetime
from pytz import timezone
from dateutil.relativedelta import *

In [3]:
# 한국 시간
KST = dt.today() + relativedelta(hours=9)
print(f"Start job time: {KST}")

Start job time: 2023-03-20 09:50:09.574873


In [4]:
def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

keychain = json.loads(get_secret())
ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']

BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']

S3_PATH_REUTER = keychain['S3_PATH_REUTER']
S3_PATH_WWO = keychain['S3_PATH_WWO']
S3_PATH_STAGE = keychain['S3_PATH_STAGE']
S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']

region = 'ap-northeast-2'
boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                              aws_secret_access_key = ACCESS_SECRET_KEY,
                              region_name = region)
sm_session = sagemaker.Session(boto_session = boto3_session)

s3_resource = boto3_session.resource('s3')
palmoil_bucket = s3_resource.Bucket(BUCKET_NAME_USECASE)
datalake_bucket = s3_resource.Bucket(DATALAKE_BUCKET_NAME)

sm_client = boto3_session.client('sagemaker')
qs_client = boto3_session.client('quicksight')
s3_client = boto3_session.client('s3')
sts_client = boto3_session.client("sts")

In [12]:
!aws s3 ls s3://crude-palm-oil-prices-forecast/golden-data/2023/03/19/1679292475.0/

                           PRE scaler-files/
                           PRE test/
                           PRE train/


In [29]:
model_list[0]

{'ModelPackageGroupName': 'crude-palm-oil-prices-forecast',
 'ModelPackageVersion': 37,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:model-package/crude-palm-oil-prices-forecast/37',
 'ModelPackageDescription': 'Autogluon',
 'CreationTime': datetime.datetime(2023, 3, 19, 21, 45, 21, 345000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'PendingManualApproval'}

In [36]:
model_list = sm_client.list_model_packages(ModelPackageGroupName = 'crude-palm-oil-prices-forecast')['ModelPackageSummaryList']
model_spec = sm_client.describe_model_package(ModelPackageName=model_list[0]['ModelPackageArn'])
print(model_spec['CustomerMetadataProperties']['champion_model'])
print(model_spec['CustomerMetadataProperties']['train_data'])
print(model_spec['CustomerMetadataProperties']['test_data'])

Naive
s3://crude-palm-oil-prices-forecast/golden-data/2023/03/19/1679292475.0/train
s3://crude-palm-oil-prices-forecast/golden-data/2023/03/19/1679292475.0/test


In [38]:
train_df = pd.read_csv(f"{model_spec['CustomerMetadataProperties']['train_data']}/train_fold1.csv")
test_df = pd.read_csv(f"{model_spec['CustomerMetadataProperties']['test_data']}/test_fold1.csv")
sum_df = pd.concat([train_df, test_df]).reset_index(drop = True)
sum_df

,ds,high,low,open,y,ric
0,2018-01-02,0.124679,0.130934,0.128052,0.127877,BOc1
1,2018-01-03,0.129811,0.135588,0.128205,0.133669,BOc1
2,2018-01-04,0.129811,0.138691,0.133089,0.133364,BOc1
3,2018-01-05,0.131472,0.139932,0.133547,0.131992,BOc1
4,2018-01-08,0.128906,0.134037,0.130189,0.128639,BOc1
...,...,...,...,...,...,...
120951,2023-03-13,0.313766,0.313750,0.308679,0.308784,Wc3
120952,2023-03-14,0.318236,0.325937,0.309872,0.321798,Wc3
120953,2023-03-15,0.337306,0.358125,0.326275,0.339840,Wc3
120954,2023-03-16,0.340286,0.360312,0.332389,0.343094,Wc3


In [62]:
%%writefile src/v1.2/prediction-autogluon.py

import argparse
import os
import requests
import tempfile
import subprocess, sys
import json

import glob
import pandas as pd
import joblib
import pickle
import tarfile
from io import StringIO, BytesIO

import logging
import logging.handlers

import time
from datetime import datetime as dt

import boto3


###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()

def get_bucket_key_from_uri(uri):
    uri_aws_path = uri.split('//')[1]
    uri_bucket = uri_aws_path.rsplit('/')[0]
    uri_file_path = '/'.join(uri_aws_path.rsplit('/')[1:])
    return uri_bucket, uri_file_path

def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret
        
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--output_dir', type = str, default = "/opt/ml/processing/output", help='예측 결과값이 저장되는 곳, Inference 결과가 저장된다.')
    parser.add_argument('--ml_algorithm_name', type=str, default = 'Autogluon')  
    parser.add_argument('--model_package_group_name', type=str, default = BUCKET_NAME_USECASE)  
    parser.add_argument('--qs_data_name', type=str, default = 'forecast result')    

    return parser.parse_args()
        
if __name__=='__main__':
    ########################################### 
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'autogluon==0.6.1'])
    from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
    
    ############################################
    ###### Secret Manager에서 키값 가져오기  #######
    ########################################### 
    logger.info(f"\n### Loading Key value from Secret Manager")
    keychain = json.loads(get_secret())
    ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
    ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']
    BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
    DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']
    S3_PATH_REUTER = keychain['S3_PATH_REUTER']
    S3_PATH_WWO = keychain['S3_PATH_WWO']
    S3_PATH_STAGE = keychain['S3_PATH_STAGE']
    S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
    S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
    S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']
    
    boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                                  aws_secret_access_key = ACCESS_SECRET_KEY,
                                  region_name = 'ap-northeast-2')
    
    s3_client = boto3_session.client('s3')
    sm_client = boto3_session.client('sagemaker')
    qs_client = boto3_session.client('quicksight')

    sts_client = boto3_session.client("sts")
    user_account_id = sts_client.get_caller_identity()["Account"]
    ######################################
    ## 커맨드 인자, Hyperparameters 처리 ##
    ######################################  
    logger.info("######### Argument Info ####################################")
    logger.info("### start training code")    
    logger.info("### Argument Info ###")
    args = parse_args()             
    logger.info(f"args.output_dir: {args.output_dir}")
    logger.info(f"args.ml_algorithm_name: {args.ml_algorithm_name}")
    logger.info(f"args.model_package_group_name: {args.model_package_group_name}")
    logger.info(f"args.qs_data_name: {args.qs_data_name}")
# prediction_output_path = f"s3://crude-palm-oil-prices-forecast/predicted-data/2023/03/19/1679292475.0/result"

    output_dir = args.output_dir
    ml_algorithm_name = args.ml_algorithm_name
    model_package_group_name = args.model_package_group_name
    qs_data_name = args.qs_data_name

    model_dir = f'{output_dir}/model'
    os.makedirs(model_dir, exist_ok=True)
    
    result_dir = f'{output_dir}/result'
    os.makedirs(result_dir, exist_ok=True)

    manifest_base_path = f'{output_dir}/manifest'
    os.makedirs(manifest_base_path, exist_ok=True)
    ##########################################################
    ###### 적합한 모델의 URI 찾고, 탑 성능 모델 이름 가져오기 ##########
    #########################################################
    logger.info("\n######### Finding suitable model uri ####################################")
    model_registry_list = sm_client.list_model_packages(ModelPackageGroupName = model_package_group_name)['ModelPackageSummaryList']
    for model in model_registry_list:
        if (model['ModelPackageGroupName'] == model_package_group_name and
            model['ModelApprovalStatus'] == 'Approved' and
            model['ModelPackageDescription'] == ml_algorithm_name):
            mr_arn = model['ModelPackageArn']
            break
    model_spec = sm_client.describe_model_package(ModelPackageName=mr_arn)
    train_data_dir = model_spec['CustomerMetadataProperties']['train_data']
    test_data_dir = model_spec['CustomerMetadataProperties']['test_data']
    
    s3_model_uri = model_spec['InferenceSpecification']['Containers'][0]['ModelDataUrl']
    champion_model = model_spec['CustomerMetadataProperties']['champion_model']

    logger.info(f"Found suitable model uri: {s3_model_uri}")
    logger.info(f"And top model name: {champion_model}")
    
    logger.info("\n#########Download suitable model file  ####################################")
    model_bucket, model_key = get_bucket_key_from_uri(s3_model_uri)  
    logger.info(f"\nmodel_bucket: {model_bucket}, model_key: {model_key}  ####################################")
    model_obj = s3_client.get_object(Bucket = model_bucket, Key = model_key)
    
    ##########################################################
    ###### 모델 압축 풀고 TimeseriesDataFrame으로 변환 ##########
    #########################################################
    logger.info("\n######### Model zip file extraction ####################################")
    with tarfile.open(fileobj=model_obj['Body'], mode='r|gz') as file:
        file.extractall(output_dir)
    logger.info(f"list in {model_dir}: {os.listdir(model_dir)}")
    logger.info("\n######### Convert df_test dataframe into TimeSeriesDataFrame  ###########")        
    df_train = pd.read_csv(os.path.join(train_data_dir, 'train_fold1.csv'))
    df_test = pd.read_csv(os.path.join(test_data_dir, 'test_fold1.csv'))
    sum_df = pd.concat([df_train, df_test]).reset_index(drop = True)
    sum_df.loc[:, "ds"] = pd.to_datetime(sum_df.loc[:, "ds"])
    
    tdf_train = TimeSeriesDataFrame.from_data_frame(
        sum_df,
        id_column="ric",
        timestamp_column="ds",
    )
    logger.info(f"sum_df sample: head(2) \n {sum_df.head(2)}")
    logger.info(f"sum_df sample: tail(2) \n {sum_df.tail(2)}")

    ################################
    ###### Prediction 시작 ##########
    ###############################
    logger.info("\n######### Start prediction  ###########")        
    loaded_trainer = pickle.load(open(f"{model_dir}/models/trainer.pkl", 'rb'))
    logger.info(f"loaded_trainer: {loaded_trainer}")
    prediction_ag_model = loaded_trainer.predict(data = tdf_train,
                                                 model = champion_model)
    logger.info(f"prediction_ag_model sample: head(2) \n {prediction_ag_model.head(2)}")
    prediction_result = prediction_ag_model.loc['FCPOc3']
    logger.info(f"prediction_ag_model sample: head(2) \n {prediction_result.head(2)}")
    prediction_result.to_csv(f'{result_dir}/prediction_result.csv')

Overwriting src/v1.2/prediction-autogluon.py


In [59]:
!aws s3 cp 'src/v1.2/prediction-autogluon.py' 's3://crude-palm-oil-prices-forecast/src/prediction-autogluon.py' --exclude ".ipynb_checkpoints*"

upload: src/v1.2/prediction-autogluon.py to s3://crude-palm-oil-prices-forecast/src/prediction-autogluon.py


In [41]:
prediction_code = 's3://crude-palm-oil-prices-forecast/src/prediction-autogluon.py'
%store prediction_code

Stored 'prediction_code' (str)


In [42]:
%store

Stored variables and their in-db values:
prediction_code             -> 's3://crude-palm-oil-prices-forecast/src/predictio


In [10]:
%store -r

# 1. 모델 빌딩 파이프라인 의 스텝(Step) 생성
## 1) 모델 빌딩 파이프라인 변수 생성
파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.

지원되는 파라미터 타입은 다음과 같습니다:

- ParameterString - 파이썬 타입에서 str
- ParameterInteger - 파이썬 타입에서 int
- ParameterFloat - 파이썬 타입에서 float
이들 파라미터를 정의할 때 디폴트 값을 지정할 수 있으며 파이프라인 실행시 재지정할 수도 있습니다. 지정하는 디폴트 값은 파라미터 타입과 일치하여야 합니다.

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

- processing_instance_type - 프로세싱 작업에서 사용할 ml.* 인스턴스 타입
- processing_instance_count - 프로세싱 작업에서 사용할 인스턴스 개수
- training_instance_type - 학습작업에서 사용할 ml.* 인스턴스 타입
- model_approval_status - 학습된 모델을 CI/CD를 목적으로 등록할 때의 승인 상태 (디폴트는 "PendingManualApproval")
- input_data - 입력데이터에 대한 S3 버킷 URI
파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.

## 2) 로컬에서 테스트 - 실패

In [12]:
if LOCAL_MODE:
    # 도커 컨테이너 입력 폴더: train, test data가 들어가는 부분
    base_input_dir = 'opt/ml/processing/input'
    os.makedirs(base_input_dir, exist_ok=True)

    # 도커 컨테이너 기본 출력 폴더: 예측결과가 출력되는곳
    base_output_dir = 'opt/ml/processing/output'
    os.makedirs(base_output_dir, exist_ok=True)

    model_package_group_name = bucket

'crude-palm-oil-prices-forecast'

In [16]:
# !aws s3 cp 's3://crude-palm-oil-prices-forecast/golden-data/2022/11/26/train.csv' \
#             'opt/ml/processing/input/train.csv'

download: s3://palm-oil-price-forecast/golden-data/2022/11/26/train.csv to opt/ml/processing/input/train.csv


In [17]:
# !aws s3 cp 's3://palm-oil-price-forecast/golden-data/2022/11/26/test.csv' \
#             'opt/ml/processing/input/test.csv'

download: s3://palm-oil-price-forecast/golden-data/2022/11/26/test.csv to opt/ml/processing/input/test.csv


In [ ]:
# !python src/prediction.py --base_input_path {base_input_dir} \
#                           --output_dir {base_output_dir} \
#                           --model_package_group_name {bucket} \

Traceback (most recent call last):
  File "src/prediction.py", line 23, in <module>
    from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
ModuleNotFoundError: No module named 'autogluon'


# 2. 파이프라인 정의 및 실행

In [44]:
from sagemaker.workflow.parameters import (ParameterInteger,
                                           ParameterString,
                                          )

prediction_instance_type = ParameterString(
    name = "PredctionInstanceType",
    default_value = "ml.m5.xlarge"
)
prediction_instance_count = ParameterInteger(
    name = "PredctionInstanceCount",
    default_value = 1
)

In [45]:
prediction_output_path = f"s3://crude-palm-oil-prices-forecast/predicted-data/2023/03/19/1679292475.0/result"
prediction_output_path
manifest_output_path = f"s3://crude-palm-oil-prices-forecast/predicted-data/2023/03/19/1679292475.0/manifest"

's3://crude-palm-oil-prices-forecast/predicted-data/2023/03/19/1679292475.0/result'

## 1) 스텝정의

### (2) ScriptProcessor 진행

In [64]:
from sagemaker.image_uris import retrieve
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

role = sagemaker.get_execution_role()
image_uri = retrieve(framework='mxnet',
                     region='ap-northeast-2',
                     version='1.9.0',
                     py_version='py38',
                     image_scope='training',
                     instance_type="ml.m5.xlarge")

script_processor_prediction = ScriptProcessor(
    command=['python3'],
    image_uri=image_uri,
    instance_type = prediction_instance_type,
    instance_count = prediction_instance_count,
    base_job_name = f"{BUCKET_NAME_USECASE}(Prediction)",
    role = role,
)

In [66]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_prediction = ProcessingStep(
    name = f"{BUCKET_NAME_USECASE}-Prediction",
    processor = script_processor_prediction,
    inputs=[],
    outputs=[
        ProcessingOutput(output_name = "prediction_data",
                         source = "/opt/ml/processing/output/result",
                         destination = prediction_output_path)
        ],
    job_arguments = ["--model_package_group_name", BUCKET_NAME_USECASE,
                     "--ml_algorithm_name", 'Autogluon'],
    code = prediction_code
)


### 1) 스텝 정의

### 1) 파이프라인 실행

In [67]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep

pipeline = Pipeline(name = BUCKET_NAME_USECASE,
                    parameters = [
                        prediction_instance_type,        
                        prediction_instance_count,
                    ],
                    steps=[step_prediction]
)

In [68]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'PredctionInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'PredctionInstanceCount', 'Type': 'Integer', 'DefaultValue': 1}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'crude-palm-oil-prices-forecast-Prediction',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.PredctionInstanceType'},
      'InstanceCount': {'Get': 'Parameters.PredctionInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/mxnet-training:1.9.0-cpu-py38',
     'ContainerArguments': ['--model_package_group_name',
      'crude-palm-oil-prices-forecast',
      '--ml_algorithm_name',
      'Autogluon'],
     'ContainerEntrypoint': ['python3',
      '/opt/ml/p

In [69]:
%%time
start = time.time()
pipeline.upsert(role_arn=sagemaker.get_execution_role())
execution = pipeline.start()
execution.wait() #실행이 완료될 때까지 기다린다.
end = time.time()

CPU times: user 746 ms, sys: 20 ms, total: 766 ms
Wall time: 6min 33s


In [70]:
print(f"prediction 시간 : {end - start:.1f} sec")
print(f"prediction 시간 : {((end - start)/60):.1f} min")

prediction 시간 : 393.9 sec
prediction 시간 : 6.6 min


[2022년 11월 29일]
- prediction 시간 : 423.1 sec
- prediction 시간 : 7.1 min

In [55]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:pipeline/crude-palm-oil-prices-forecast',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:pipeline/crude-palm-oil-prices-forecast/execution/nqsfnu6enbba',
 'PipelineExecutionDisplayName': 'execution-1679279948741',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'crude-palm-oil-prices-forecast',
  'TrialName': 'nqsfnu6enbba'},
 'CreationTime': datetime.datetime(2023, 3, 20, 2, 39, 8, 684000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 3, 20, 2, 45, 45, 797000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '2811dbcd-de40-42c9-8019-86778eac60f3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2811dbcd-de40-42c9-8019-86778eac60f3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '541',
   'date': 'Mon, 20 Mar 2023 04:09:15 GMT'},
  'RetryAttempts': 0}}

In [56]:
execution.list_steps()

[{'StepName': 'crude-palm-oil-prices-forecast-Prediction',
  'StartTime': datetime.datetime(2023, 3, 20, 2, 39, 10, 234000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 3, 20, 2, 45, 45, 310000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:processing-job/pipelines-nqsfnu6enbba-crude-palm-oil-price-fjbqjvagxd'}}}]

In [ ]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)